In [ ]:
%matplotlib inline


# Transfer Learning for Hyperparameter Search

**Author(s)**: Romain Egele.


In [ ]:
import functools


def run(config: dict, N: int) -> float:
    y = -sum([config[f"x{i}"] ** 2 for i in range(N)])
    return y


run_small = functools.partial(run, N=1)
run_large = functools.partial(run, N=2)

In [ ]:
from deephyper.problem import HpProblem


N = 1
problem_small = HpProblem()
for i in range(N):
    problem_small.add_hyperparameter((-10.0, 10.0), f"x{i}")
problem_small

In [ ]:
N = 2
problem_large = HpProblem()
for i in range(N):
    problem_large.add_hyperparameter((-10.0, 10.0), f"x{i}")
problem_large

In [ ]:
from deephyper.evaluator import Evaluator
from deephyper.evaluator.callback import TqdmCallback
from deephyper.search.hps import CBO

results = {}
max_evals = 20
evaluator_small = Evaluator.create(
    run_small, method="serial", method_kwargs={"callbacks": [TqdmCallback(max_evals)]}
)
search_small = CBO(problem_small, evaluator_small, random_state=42)
results["Small"] = search_small.search(max_evals)

In [ ]:
evaluator_large = Evaluator.create(
    run_large, method="serial", method_kwargs={"callbacks": [TqdmCallback(max_evals)]}
)
search_large = CBO(problem_large, evaluator_large, random_state=42)
results["Large"] = search_large.search(max_evals)

In [ ]:
evaluator_large_tl = Evaluator.create(
    run_large, method="serial", method_kwargs={"callbacks": [TqdmCallback(max_evals)]}
)
search_large_tl= CBO(problem_large, evaluator_large_tl, random_state=42)
search_large_tl.fit_generative_model(results["Large"])
results["Large+TL"] = search_large_tl.search(max_evals)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()

for strategy, df in results.items():
    x = [i for i in range(len(df))]
    plt.scatter(x, df.objective, label=strategy)
    plt.plot(x, df.objective.cummax())

plt.xlabel("Time (sec.)")
plt.ylabel("Objective")
plt.grid()
plt.legend()
plt.show()